In [1]:
from pathlib import Path
from utils.get_input import get_input

# inp = Path("./test_input").read_text()
inp = get_input(5)
inp;

In [ ]:
# %%timeit
rules, updates = inp.split("\n\n")

updates = [[int(xx) for xx in x.split(",")] for x in updates.split("\n")]

rules = [[int(xx) for xx in x.split("|")] for x in rules.split("\n")]

rules_map_k_before_v = {}
for r1, r2 in rules:
    if r1 in rules_map_k_before_v:
        rules_map_k_before_v[r1].append(r2)
    else:
        rules_map_k_before_v[r1] = [r2]

rules_map_v_before_k = {}
for k, v in rules_map_k_before_v.items():
    for i in v:
        if i in rules_map_v_before_k:
            rules_map_v_before_k[i].append(k)
        else:
            rules_map_v_before_k[i] = [k]

In [47]:
def part1():
    sum = 0

    for update in updates:
        # print(update)
        past = set()
        whole_row = set(update)
        rules_followed = False

        for p in update:
            should_before_p = (
                rules_map_v_before_k[p] if p in rules_map_v_before_k else []
            )
            should_before_p = set(should_before_p).intersection(whole_row)
            rules_followed = set(should_before_p).issubset(past)
            if not rules_followed:
                # print("RULE MISSED", p, should_before_p, past)
                break
            # print(p, rules_followed)
            past.add(p)

        if rules_followed:
            # print("YES")
            sum += update[len(update) // 2]

    return sum


part1()

7307

In [48]:
%%timeit
part1()

1.73 ms ± 19.9 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
def part2():
    sum = 0

    for update in updates:
        # print("\nUPDATE: ", update)
        past = set()
        whole_row = set(update)

        rules_followed_all = True
        i = 0
        while i < len(update):

            p = update[i]
            should_before_p = (
                rules_map_v_before_k[p] if p in rules_map_v_before_k else []
            )
            should_before_p = set(should_before_p).intersection(whole_row)
            rules_followed = set(should_before_p).issubset(past)
            past.add(p)
            if not rules_followed:
                rules_followed_all = False
                # print("RULE MISSED", p, should_before_p, past)
                move_before_p = should_before_p - past
                # print(f"Moving {move_before_p} before {p}")
                update = (
                    update[:i]
                    + list(move_before_p)
                    + [x for x in update[i:] if x not in move_before_p]
                )
                # print("NEW UPDATE", update)
                i = 0
                past = set()
            # print(p, rules_followed)

            i += 1

        if not rules_followed_all:
            to_add = update[len(update) // 2]
            # print("YES, adding", to_add)
            sum += to_add

    return sum


part2()

4713

In [51]:
%%timeit
part2()

5.73 ms ± 143 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
